<a href="https://colab.research.google.com/github/Salma-Jamal/Movie-Recommendation-System/blob/main/models/Movie_Recommendation_System_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Input, Flatten, Dot, Dense
from keras.models import Model
from keras.layers import Embedding, Input, Flatten, Dot, Concatenate, Dense
from keras.models import Model
import numpy as np

ratings = pd.read_csv("/content/drive/MyDrive/partition500k.csv")

# Define Dictionary to convert Genre Column

In [28]:
genres_dict = {'Action': 0,
 'Comedy': 1,
 'Drama': 2,
 'Adventure': 3,
 'Crime': 4,
 'Horror': 5,
 'Animation': 6,
 'Children': 7,
 'Thriller': 8,
 'Documentary': 9,
 'Sci-Fi': 10,
 'Mystery': 11,
 'Fantasy': 12,
 'Musical': 13,
 'Film-Noir': 14,
 'Western': 15,
 'Romance': 16,
 'War': 17,
 'IMAX': 18}

In [29]:
ratings['genres_edit'] = ratings['genres'].map(lambda x: x.split('|')[0])

In [30]:
ratings['genres_edit'] = ratings['genres_edit'].map(lambda x:genres_dict[x])

In [31]:
ratings

,userId,movieId,rating,timestamp,title,genres,genres_edit
0,1,122,5.0,1996-08-02,Boomerang (1992),Comedy|Romance,1
1,1,185,5.0,1996-08-02,"Net, The (1995)",Action|Crime|Thriller,0
2,1,231,5.0,1996-08-02,Dumb & Dumber (1994),Comedy,1
3,1,292,5.0,1996-08-02,Outbreak (1995),Action|Drama|Sci-Fi|Thriller,0
4,1,316,5.0,1996-08-02,Stargate (1994),Action|Adventure|Sci-Fi,0
...,...,...,...,...,...,...,...
499995,3810,4369,2.5,2006-11-13,"Fast and the Furious, The (2001)",Action|Crime|Thriller,0
499996,3810,4370,2.0,2006-11-13,A.I. Artificial Intelligence (2001),Adventure|Drama|Sci-Fi,3
499997,3810,4378,4.0,2007-01-14,Sexy Beast (2000),Comedy|Crime|Drama,1
499998,3810,4396,1.0,2006-11-28,"Cannonball Run, The (1981)",Action|Comedy,0


# Training

In [32]:
train, test = train_test_split(ratings, test_size=0.2, random_state=42)

# inputs for the model
user_input = Input(shape=[1])
movie_input = Input(shape=[1])
genre_input = Input(shape=[1])

# create the embedding layer for the model feautres (inputs)
user_embedding = Embedding(input_dim=len(ratings.userId.unique()), output_dim=5)(user_input)
user_vec = Flatten()(user_embedding)


movie_embedding = Embedding(input_dim=len(ratings.movieId.unique()), output_dim=5)(movie_input)
movie_vec = Flatten()(movie_embedding)


genre_embedding = Embedding(input_dim=len(ratings.genres_edit.unique()), output_dim=5)(genre_input)
genre_vec = Flatten()(genre_embedding)


# Concatenate the embeddings
concat = Concatenate()([user_vec, movie_vec,genre_vec])

# output layer
output = Dense(1, activation='sigmoid')(concat)

# model
model = Model([user_input, movie_input,genre_input], output)

# compile it 
model.compile(loss='mean_squared_error', optimizer='adam')

# fit to the training data
model.fit([train.userId, train.movieId,train.genres_edit], train.rating, epochs=100, verbose=1)

# evaluate the model on the test data
model.evaluate([test.userId, test.movieId,test.genres_edit], test.rating)


Epoch 1/100
12500/12500 [==============================] - 43s 3ms/step - loss: 7.6372
Epoch 2/100
12500/12500 [==============================] - 41s 3ms/step - loss: 7.5746
Epoch 3/100
12500/12500 [==============================] - 41s 3ms/step - loss: 7.5746
Epoch 4/100
12500/12500 [==============================] - 41s 3ms/step - loss: 7.5746
Epoch 5/100
12500/12500 [==============================] - 41s 3ms/step - loss: 7.5746
Epoch 6/100
12500/12500 [==============================] - 41s 3ms/step - loss: 7.5746
Epoch 7/100
12500/12500 [==============================] - 41s 3ms/step - loss: 7.5746
Epoch 8/100
12500/12500 [==============================] - 41s 3ms/step - loss: 7.5746
Epoch 9/100
12500/12500 [==============================] - 41s 3ms/step - loss: 7.5746
Epoch 10/100
12500/12500 [==============================] - 41s 3ms/step - loss: 7.5746
Epoch 11/100
12500/12500 [==============================] - 41s 3ms/step - loss: 7.5746
Epoch 12/100
12500/12500 [===============

7.543327331542969

#Evaluate

In [44]:
model.evaluate([test.userId, test.movieId,test.genres_edit], test.rating)

3125/3125 [==============================] - 5s 2ms/step - loss: 7.5433


7.543327331542969

# Recommend a movie for a user

In [58]:
def RecommendMovie(id):
  # get the predicted ratings for all movies for a specific user
  # Select a User
  user_id = id

  # Get Unique movie Ids
  movie_ids = np.array(ratings.movieId.unique())

  # Get Genres of each movie
  movie_genres =  []
  for i in ratings.movieId.unique():
    movie_genres.append(ratings[ratings['movieId'] == i]['genres_edit'].tolist()[0])
  movie_genres = np.array(movie_genres)

  # Predict user rating for each unique movie
  user_ratings = model.predict([np.array([user_id]*len(movie_ids)), movie_ids,movie_genres])

  # sort by predicted rating and recommend the top N movies
  top_movies = movie_ids[np.argsort(-user_ratings.flatten())]
  return top_movies

In [60]:
user_id = 7
top_movies = RecommendMovie(user_id)

print("Movies Recommended for User",user_id,":")
for i in top_movies[:10]:
  print("Movie id:",i,"-----",ratings[ratings['movieId'] == i]['title'].tolist()[0])

273/273 [==============================] - 0s 1ms/step
Movies Recommended for User 7 :
Movie id: 122 ----- Boomerang (1992)
Movie id: 43744 ----- Imagine Me & You (2005)
Movie id: 42021 ----- White Countess, The (2005)
Movie id: 42007 ----- Rumor Has It... (2005)
Movie id: 42004 ----- Transamerica (2005)
Movie id: 41025 ----- Good Woman, A (2004)
Movie id: 40966 ----- Libertine, The (2004)
Movie id: 40955 ----- Breakfast on Pluto (2005)
Movie id: 39449 ----- Prime (2005)
Movie id: 39435 ----- Legend of Zorro, The (2005)


In [61]:
user_id = 55
top_movies = RecommendMovie(user_id)

print("Movies Recommended for User",user_id,":")
for i in top_movies[:10]:
  print("Movie id:",i,"-----",ratings[ratings['movieId'] == i]['title'].tolist()[0])

273/273 [==============================] - 0s 1ms/step
Movies Recommended for User 55 :
Movie id: 122 ----- Boomerang (1992)
Movie id: 6927 ----- Human Stain, The (2003)
Movie id: 6832 ----- Regarding Henry (1991)
Movie id: 6763 ----- Duplex (2003)
Movie id: 6687 ----- My Boss's Daughter (2003)
Movie id: 6493 ----- Alex and Emma (2003)
Movie id: 6202 ----- Late Marriage (Hatuna Meuheret) (2001)
Movie id: 6060 ----- Guru, The (2002)
Movie id: 6041 ----- Amen. (2002)
Movie id: 5817 ----- Ararat (2002)


In [62]:
user_id = 5000
top_movies = RecommendMovie(user_id)

print("Movies Recommended for User",user_id,":")
for i in top_movies[:10]:
  print("Movie id:",i,"-----",ratings[ratings['movieId'] == i]['title'].tolist()[0])

273/273 [==============================] - 0s 1ms/step
Movies Recommended for User 5000 :
Movie id: 122 ----- Boomerang (1992)
Movie id: 63113 ----- Quantum of Solace (2008)
Movie id: 62394 ----- Max Payne (2008)
Movie id: 62113 ----- How to Lose Friends & Alienate People (2008)
Movie id: 61132 ----- Tropic Thunder (2008)
Movie id: 59421 ----- What Happens in Vegas... (2008)
Movie id: 59387 ----- Fall, The (2006)
Movie id: 59037 ----- Speed Racer (2008)
Movie id: 58655 ----- Drillbit Taylor (2008)
Movie id: 58347 ----- Penelope (2006)


In [63]:
user_id = 900
top_movies = RecommendMovie(user_id)

print("Movies Recommended for User",user_id,":")
for i in top_movies[:10]:
  print("Movie id:",i,"-----",ratings[ratings['movieId'] == i]['title'].tolist()[0])

273/273 [==============================] - 0s 1ms/step
Movies Recommended for User 900 :
Movie id: 122 ----- Boomerang (1992)
Movie id: 42004 ----- Transamerica (2005)
Movie id: 41025 ----- Good Woman, A (2004)
Movie id: 40966 ----- Libertine, The (2004)
Movie id: 40955 ----- Breakfast on Pluto (2005)
Movie id: 39449 ----- Prime (2005)
Movie id: 39435 ----- Legend of Zorro, The (2005)
Movie id: 38499 ----- Angels in America (2003)
Movie id: 36537 ----- Thumbsucker (2005)
Movie id: 34523 ----- Chumscrubber, The (2005)
